In [8]:
import omegaconf

import os

import pandas as pd
import tiktoken

import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType, 
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch


In [35]:
INPUT_DIR = "./inputs/"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"
CONF = omegaconf.OmegaConf.load("config/api.yaml")

# parquet files generated from indexing pipeline
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 0

In [36]:
llm = ChatOpenAI(
    api_base=CONF.api.api_base, #CONF.api.endpoint,
    api_key=CONF.api.key,
    model=CONF.api.model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=2,
    api_version=CONF.api.api_version
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [37]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 18


/home/azureuser/ashutosh/miniconda3/envs/p310/lib/python3.10/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/home/azureuser/ashutosh/miniconda3/envs/p310/lib/python3.10/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,"# Dulce, Alien Data, and Intergalactic Contact...",1,7.5,"Dulce, Alien Data, and Intergalactic Contact",The impact severity rating is high due to the ...,"The community revolves around Dulce, a locatio...",[{'explanation': 'Dulce is the central entity ...,"{\n ""title"": ""Dulce, Alien Data, and Interg...",59a5cee0-35bf-4a30-8a5e-912abd27b17c
1,11,# The Signal and its Impact on Humanity\n\nThe...,1,7.5,The Signal and its Impact on Humanity,The impact severity rating is high due to the ...,"The community revolves around The Signal, a pa...",[{'explanation': 'The Signal is the central en...,"{\n ""title"": ""The Signal and its Impact on ...",0a37b9a6-efaf-4d92-8226-631c28ef6ca0
2,12,# Earth and The Terminal\n\nThe community revo...,1,6.5,Earth and The Terminal,The impact severity rating is moderate due to ...,The community revolves around Earth and The Te...,[{'explanation': 'Earth is the central entity ...,"{\n ""title"": ""Earth and The Terminal"",\n ...",d294ab13-775c-4387-995d-c1601bef7b72
3,13,# The Paranormal Military Squad and Earth's Bi...,1,7.5,The Paranormal Military Squad and Earth's Bid ...,The impact severity rating is high due to the ...,The community revolves around The Paranormal M...,[{'explanation': 'The Paranormal Military Squa...,"{\n ""title"": ""The Paranormal Military Squad...",8c8aee13-5b41-46ff-8cb4-0cb056968162
4,15,# Paranormal Military Squad and Operation: Dul...,1,7.5,Paranormal Military Squad and Operation: Dulce,The impact severity rating is high due to the ...,The community revolves around the Paranormal M...,[{'explanation': 'Dulce Base serves as the cen...,"{\n ""title"": ""Paranormal Military Squad and...",f019aabc-3e4b-4c1e-94a6-e1c98116e192


In [38]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [39]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [40]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)



In [41]:
result = await search_engine.asearch(
    "What is the major conflict in this story and who are the protagonist and antagonist?"
)

print(result.response)

### Major Conflict

The central conflict in the story revolves around **Operation: Dulce**, a mission led by Alex Mercer and the Paranormal Military Squad. The primary objective of this operation is to decrypt an alien code and establish communication with an unknown intelligence. This mission is fraught with significant challenges and potential risks, as the team must navigate the complexities of the unknown and decipher the intentions of the alien intelligence [Data: Reports (0, 1, 3)].

### Protagonist

The protagonist of the story is **Alex Mercer**. He is the leader of the team and the Paranormal Military Squad. Alex displays determination, leadership, and diplomatic skills throughout the mission to decrypt the alien code and make contact with the unknown intelligence [Data: Reports (0, 3)].

### Antagonist

The antagonist in the story is the **unknown alien intelligence**. The team's mission to understand and communicate with this intelligence presents significant challenges and 

In [42]:
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,3,The Team and Operation: Dulce,1.000000,# The Team and Operation: Dulce\n\nThe communi...,6.0
1,1,Operation: Dulce and the Unknown Intelligence,0.964286,# Operation: Dulce and the Unknown Intelligenc...,7.5
2,0,Operation: Dulce and the Alien Code,0.821429,# Operation: Dulce and the Alien Code\n\nThe c...,7.5
3,2,The Signal and Paranormal Military Squad,0.250000,# The Signal and Paranormal Military Squad\n\n...,7.5
4,4,Operation: Dulce and the Alien Intelligence,0.214286,# Operation: Dulce and the Alien Intelligence\...,7.5


In [43]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 2. LLM tokens: 5543
